In [9]:
import os
import time
import cv2
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
from shapely.ops import unary_union
from ultralytics import YOLO
from joblib import Parallel, delayed

In [10]:
model = YOLO('bestY8.pt')
files = 'тест'
allowed_ext = ('.jpg', '.jpeg', '.png')
reference_csv = 'groups_count.csv'
folder_path = 'тест/'

In [11]:
#Преобразуем вершины боксов в полноценные полигоны
def rect_to_polygon(row):
    x1, y1, x2, y2 = row['x1'], row['y1'], row['x2'], row['y2']
    return Polygon([(x1, y1), (x2, y1), (x2, y2), (x1, y2)])

In [17]:
#Функция для запуска yolo
def run_yolo_inference(file_path):
    img = cv2.imread(file_path)
    result = model(img, verbose = False)[0]
    xyxy = result.boxes.xyxy.cpu().numpy()
    cls = result.boxes.cls.cpu().numpy()
    return {
        'filename': os.path.basename(file_path),
        'detections': list(zip(xyxy, cls))
    }
#Сам алгоритм группировки
def group_boxes(detection_info):
    filename = detection_info['filename']
    detections = detection_info['detections']
#Формируем данные
    data = []
    for xc in detections:
        box, class_id = xc
        data.append({
            'filename': filename,
            'class_id': int(class_id),
            'x1': int(box[0]),
            'y1': int(box[1]),
            'x2': int(box[2]),
            'y2': int(box[3]),
            'center': ((box[0] + box[2]) / 2, (box[1] + box[3]) / 2)
        })

    df = pd.DataFrame(data)
    df['geometry'] = df.apply(rect_to_polygon, axis=1)
    gdf = gpd.GeoDataFrame(df, geometry='geometry')
    gdf['square'] = gdf.geometry.area
    #group_id это счетчик групп, чтобы по ходу выполнения присваивались уникальные
    group_gdf = gdf.copy()
    group_gdf['group']=np.nan
    group_id = 1

    id_list = []
    #Функция назначения превоначальных групп
    def assign_group(row):
        nonlocal group_id
        nonlocal id_list
        if row['class_id'] in [0, 2]:
            row['group'] = group_id
            group_id += 1
            id_list.append(row.name)
        return row['group']
        
    group_gdf['group'] = group_gdf.apply(assign_group, axis =1 )
    #Тут я сразу разбил на тех у кого есть группа и нет, чтобы не бегать по фрейму туда сюда
    unlabeled = group_gdf[group_gdf['class_id'] == 1].copy()
    grouped = group_gdf.loc[id_list][['geometry', 'group']].copy()
    next_group_id = group_id
   #Здесь либо присоединяем 1 к группе, либо если он 1, то даем ему новую
    def find_best_group_grouped(current_geom, grouped):
        intersection_areas = grouped['geometry'].apply(lambda geom: current_geom.intersection(geom).area if not current_geom.intersection(geom).is_empty else 0)
        nonlocal next_group_id
        if intersection_areas.max() > 0:
            best_match_idx = intersection_areas.idxmax()  
            return grouped.loc[best_match_idx, 'group']
        else:
            next_group_id+=1
            return next_group_id-1

    unlabeled['group'] = unlabeled['geometry'].apply(lambda geom: find_best_group_grouped(geom, grouped))

    group_gdf.update(unlabeled[['group']])
    #Удалим группы с только одним объектом и если это объект 0 или 2
    group_counts = group_gdf['group'].value_counts()
    single_occurrence_groups = group_counts[group_counts == 1].index
    condition_to_drop = group_gdf['group'].isin(single_occurrence_groups) & group_gdf['class_id'].isin([0, 2])
    group_gdf = group_gdf[~condition_to_drop]
    
    
    return filename, group_gdf['group'].nunique()



In [15]:

file_list = [os.path.join(folder_path, filename) for filename in os.listdir(folder_path)
             if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff'))]


start_gpu = time.time()
detections = [run_yolo_inference(file_path) for file_path in file_list]
end_gpu = time.time()

print(f"GPU инференс времени: {end_gpu - start_gpu:.2f} секунд")
print(f"GPU инференс времени: {(end_gpu - start_gpu)/104} секунд")


start_cpu = time.time()
group_results = Parallel(n_jobs=-1)(delayed(group_boxes)(detection) for detection in detections)
end_cpu = time.time()

print(f"CPU группировка времени: {end_cpu - start_cpu:.2f} секунд")
print(f"Среднее CPU группировка времени: {(end_cpu - start_cpu)/104} секунд")


end_df = pd.DataFrame(group_results, columns=['filename', 'group_count'])

GPU инференс времени: 9.48 секунд
GPU инференс времени: 0.09113808320118831 секунд
CPU группировка времени: 0.45 секунд
Среднее CPU группировка времени: 0.004348656305899987 секунд


In [16]:
total_error =0 
total_target_sum=0
csv_file = 'groups_count.csv'


groups_df = pd.read_csv(csv_file)
merged_df = pd.merge(end_df, groups_df, on='filename', suffixes=('_pred', '_true'))


for idx,row in merged_df.iterrows():
    error = abs(row['group_count_pred']-row['group_count_true'])
    total_error+=error
    total_target_sum+=row['group_count_true']

forecast_accuracy = 1 - (total_error/total_target_sum)

print(f"Точность по FA:{forecast_accuracy}")
print(total_error)
print(total_target_sum)

Точность по FA:0.8175519630484989
158
866


In [261]:
merged_df.to_csv('clear_result.csv', index=False)